In [33]:
#%cd ..
import json
import utils
import os
import pandas as pd
import yfinance as yf
from itables import show

if not os.path.exists('sp500.json'):
    sp500 = utils.spy_components()
    with open('./sp500.json', 'w') as f:
        json.dump(sp500, f, indent=2)
else:
    with open("./sp500.json", 'r') as f:
        sp500 = json.load(f)


In [36]:
import os
SAVE_DIR='data'
file = 'sp500_daily_prices.parquet'
file_path = os.path.join(SAVE_DIR, file)

download = True
if os.path.exists(file_path): 
    answer = input("Re-download SP500? [yes/no]")
    download = True if answer[0].lower() == 'y' else False 
if download:
    daily_df = yf.download(sp500).swaplevel(axis=1)
    daily_df.to_parquet(file_path)
else:
    daily_df = pd.read_parquet(file_path)


Re-download SP500? [yes/no] N


[*********************100%***********************]  504 of 504 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 1926-04-23 -> 2025-03-29)')


In [40]:
daily_df['AMZN']

Price,Close,High,Low,Open,Volume
Date,,,,,
1962-01-02,NaN,NaN,NaN,NaN,NaN
1962-01-03,NaN,NaN,NaN,NaN,NaN
1962-01-04,NaN,NaN,NaN,NaN,NaN
1962-01-05,NaN,NaN,NaN,NaN,NaN
1962-01-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-03-24,203.259995,203.639999,199.949997,200.000000,41625400.0
2025-03-25,205.710007,206.210007,203.220001,203.600006,31171200.0
2025-03-26,201.130005,206.009995,199.929993,205.839996,32991000.0
